In [1]:
from __future__ import print_function
from __future__ import division

import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import os
import datetime
import sys
import random

In [2]:
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split, cross_val_score
import socket
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from bayes_opt import bayesian_optimization

import statsmodels.api as sm
import sklearn.model_selection as cv
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None, verbose=0):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func, verbose=verbose)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds, verbose=verbose)
    gs.fit(X, y)
    print("BEST", gs.best_params_, gs.best_score_, gs.cv_results_, gs.scorer_)
    print("Best score: ", gs.best_score_)
    best = gs.best_estimator_
    return best

## NYC Taxi Data

#### Download the Trip Record Data

In [4]:
# For downloading TLC data for multiple months in the same year
'''
for month in range(1,2):
    urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/trip+data/"+ \
                               "yellow_tripdata_2018-{0:0=2d}.csv".format(month), 
                               "nyc.2018-{0:0=2d}.csv".format(month))
'''

# For downloading TLC data for a single month in multiple years
'''
month = 1
for year in range(2017, 2020):
    urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/trip+data/"+ \
                               "yellow_tripdata_{0:4d}-{1:0=2d}.csv".format(year, month), 
                               "nyc.{0:4d}-{1:0=2d}.csv".format(year, month))
'''

'\nmonth = 1\nfor year in range(2017, 2020):\n    urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/trip+data/"+                                "yellow_tripdata_{0:4d}-{1:0=2d}.csv".format(year, month), \n                               "nyc.{0:4d}-{1:0=2d}.csv".format(year, month))\n'

In [5]:
n = 500
nyc_df = pd.read_csv("datasets/nyc.2017-01.csv", parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
nyc_df = nyc_df.set_index('tpep_pickup_datetime')
nyc_df.describe()

#### Remove the rows that don't belong to the choosen time frame

In [ ]:
nyc_df = nyc_df.loc['2018-01']
nyc_df.head()

#### Check for null values

In [ ]:
nyc_df.isnull().sum()

#### Sort the data based on the date information and reindex it

In [ ]:
nyc_df.sort_values(by=['tpep_pickup_datetime'])
nyc_df.reset_index(inplace=True)
nyc_df.head()

#### Check for anomalies in the dataset and drop them

In [ ]:
rows_to_drop = []

for index, row in nyc_df.iterrows():
    duration = nyc_df.tpep_dropoff_datetime[index] - nyc_df.tpep_pickup_datetime[index]

    if(duration > datetime.timedelta(hours=12)):
        rows_to_drop.append(index)
        
    elif(duration.seconds != 0):
        # Miles per hour, average speed
        speed = (nyc_df.trip_distance[index]/duration.seconds)*60
        
        # Check if a taxi is going faster than they can in real life
        if((speed > 90.0) or (speed < 1.0)):
            rows_to_drop.append(index)
            
nyc_df.drop(nyc_df.index[rows_to_drop])

nyc_df = nyc_df[nyc_df['total_amount'] >= 0]
nyc_df = nyc_df[nyc_df['extra'] >= 0]
nyc_df = nyc_df[nyc_df['mta_tax'] >= 0]
nyc_df = nyc_df[nyc_df['fare_amount'] >= 0]
nyc_df = nyc_df[nyc_df['tolls_amount'] >= 0]
nyc_df = nyc_df[nyc_df['improvement_surcharge'] >= 0]

nyc_df.describe()

## NYC Taxi Zones

In [ ]:
'''
# Download the location Data
urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "taxi_zones.zip")
with zipfile.ZipFile("taxi_zones.zip","r") as zip_ref:
    zip_ref.extractall("./datasets/taxi_zones/shape")
'''

In [ ]:
def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

#### Convert shape file to data frame

In [ ]:
sf = shapefile.Reader("datasets/taxi_zones/shape/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

loc_df = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")
loc_df.head()

#### Remove location id's that are outside of the Manhattan

In [ ]:
loc_df = loc_df[loc_df.borough == "Manhattan"]
loc_df.head()

#### Convert location ids that belong to Manhattan into a list

In [ ]:
manhattan_loc_id_list = loc_df["LocationID"].tolist()
print(manhattan_loc_id_list)

## Intersect NYC Taxi and NYC Taxi Zone Data

#### Remove taxi rides that didn't originate from Manhattan

In [ ]:
nyc_df = nyc_df[nyc_df['PULocationID'].isin(manhattan_loc_id_list)]

nyc_df.head()

## Weather Data

#### Import the weather dataset

In [ ]:
weather_df = pd.read_csv("datasets/weather_data.csv")
weather_df.describe()

#### Check the types of the columns

In [ ]:
weather_df.dtypes

#### Convert events column into multiple columns

In [ ]:
weather_df['event_rain'] = 0
weather_df['event_fog'] = 0
weather_df['event_snow'] = 0

for index, row in weather_df.iterrows():
    if(isinstance(weather_df.events[index], str)):
        if("Rain" in weather_df.events[index]):
            weather_df.event_rain[index] = 1

        if("Fog" in weather_df.events[index]):
            weather_df.event_fog[index] = 1

        if("Snow" in weather_df.events[index]):
            weather_df.event_snow[index] = 1

weather_df = weather_df.drop("events", axis = 1)

#### Add primary key to be used in the NYC taxi data

In [ ]:
weather_df.insert(0, "primary_key", "") 

for index, row in weather_df.iterrows():
    key = str(row['year'])
    
    if (row['month'] == 0) or (row['month'] == 1) or (row['month'] == 2) or (row['month'] == 3) or (row['month'] == 4) or (row['month'] == 5) or (row['month'] == 6) or (row['month'] == 7) or (row['month'] == 8) or (row['month'] == 9):
        key = key + "-0" + str(row['month'])
    else:
        key = key + "-" + str(row['month'])
        
    if (row['day'] == 0) or (row['day'] == 1) or (row['day'] == 2) or (row['day'] == 3) or (row['day'] == 4) or (row['day'] == 5) or (row['day'] == 6) or (row['day'] == 7) or (row['day'] == 8) or (row['day'] == 9):
        key = key + "-0" + str(row['day'])
    else:
        key = key + "-" + str(row['day'])
        
    weather_df.primary_key[index] = key


#### Sort the data based on the date information and reindex it

In [ ]:
weather_df['primary_key'] = pd.to_datetime(weather_df['primary_key'])
weather_df = weather_df.set_index('primary_key')

#weather_df.reset_index(inplace=True)
weather_df.head()

#### Check for missing values

In [ ]:
weather_df.isnull().sum()

#### Fill rows with missing values using interpolation

In [ ]:
weather_df = weather_df.interpolate(method="linear")

#weather_df = weather_df.dropna()
weather_df.isnull().sum()

## Add Weather Data to NYC Taxi Data

In [ ]:
nyc_df['rain'] = 0
nyc_df['temperature_avg'] = 0
nyc_df['humidity_avg'] = 0
nyc_df.insert(0, "day", 0) 

for index, row in nyc_df.iterrows():
    string_key = nyc_df.tpep_pickup_datetime[index].strftime("%Y-%m-%d")
    
    row_array = weather_df.loc[string_key]
    
    nyc_df.rain[index] = row_array["precipitation"]
    nyc_df.temperature_avg[index] = row_array["temp_avg"]
    nyc_df.humidity_avg[index] = row_array["humidity_avg"]
    nyc_df.day[index] = int(nyc_df.tpep_pickup_datetime[index].strftime("%d"))


## Check for missing values

In [ ]:
nyc_df.isnull().sum()

In [ ]:
#nyc_df.tpep_pickup_datetime.count()
#nyc_df.tpep_pickup_datetime[0].strftime("%Y-%m-%d")
#weather_df.primary_key[0].strftime("%Y-%m-%d")
#weather_df.dtypes
#weather_df.loc["2016-01-10"]

nyc_df.head()

## Produce the target data for training

#### Calculate the frequency values of locations based on days

In [ ]:
temp_df = nyc_df[['day', 'PULocationID']].copy()
temp_df = temp_df.drop_duplicates(subset=['day', 'PULocationID'], keep='first')
temp_df['freq'] = 0

# Iterate over the unique location and day information
for index, row in temp_df.iterrows():
    day = temp_df.day[index]
    location_id = temp_df.PULocationID[index]
    
    # Get rows from NYC taxi data with matching days
    day_temp_df = nyc_df.loc[nyc_df['day'] == day]
    
    # Count the number of rows with matching PULocationID within the matching days
    count = len(day_temp_df.loc[day_temp_df['PULocationID'] == location_id])

    temp_df.freq[index] = count

temp_df.head()

#### Map frequency data to NYC taxi data to match the number of rows

In [ ]:
target_df = pd.DataFrame()
target_df['freq'] = 0

# Iterate over the NYC taxi data
for index, row in nyc_df.iterrows():
    day = nyc_df.day[index]
    location_id = nyc_df.PULocationID[index]
    
    day_temp_df = temp_df.loc[temp_df['day'] == day]
    location_temp = day_temp_df.loc[day_temp_df['PULocationID'] == location_id]
        
    target_df.loc[index] = location_temp.freq.values

target_df.head()

## Remove unused features before the training 

In [ ]:
nyc_df = nyc_df.drop(columns=['tpep_pickup_datetime',
                              'tpep_dropoff_datetime',
                              'DOLocationID',
                              'VendorID',
                              'RatecodeID',
                              'store_and_fwd_flag',
                              'payment_type',
                              'passenger_count',
                              'fare_amount',
                              'total_amount',
                              'trip_distance',
                              'extra',
                              'mta_tax',
                              'tip_amount',
                              'tolls_amount',
                              'improvement_surcharge'])


In [ ]:
nyc_df.dtypes

In [ ]:
target_df.dtypes

In [ ]:
#target=nycmodel[['count']]
#data=nycmodel[[col for col in nycmodel.columns if col not in ['count']]]

x_train, x_test, y_train, y_test = cv.train_test_split(nyc_df, target_df, test_size=2.0/10, random_state=5)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


## Linear Regression

In [ ]:
reg = LinearRegression().fit(x_train, y_train)

y_predictedValue = reg.predict(x_train)  
rmse = np.sqrt(mean_squared_error(y_train, y_predictedValue))
r2 = reg.score(x_train, y_train)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R^2 score is {}'.format(r2))
print("\n")

#### Random Forests

In [ ]:
# Create a Random Forest Regression estimator
estimator = RandomForestRegressor(n_estimators=20, n_jobs=-1)

# Define a grid of parameters over which to optimize the random forest
# We will figure out which number of trees is optimal
parameters = {"n_estimators": [50],
              "max_features": ["auto"], # ["auto","sqrt","log2"]
              "max_depth": [50]}
best = cv_optimize(estimator, parameters, x_train, y_train, n_folds=5, verbose=3)

In [ ]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
reg=best.fit(x_train, y_train)
training_accuracy = reg.score(x_train, y_train)
test_accuracy = reg.score(x_test, y_test)
print("############# based on standard predict ################")
print("R^2 on training data: %0.4f" % (training_accuracy))
print("R^2 on test data:     %0.4f" % (test_accuracy))

## XgBoost Regressor

In [ ]:
from random import randint
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

x_model = xgb.XGBRegressor()
param_dist = {"max_depth": [3, 4,5],
              'n_estimators': [randint(400,600)],
              "min_child_weight": [3, 4,5,6],
              "gamma":[0,0.1,0.2],
              "colsample_bytree":[0.7,0.8,0.9],
              "nthread":[3,4,5]
              }

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(x_model, param_distributions=param_dist,n_iter=n_iter_search)
random_search.fit(x_train, y_train)

In [ ]:
print(random_search.best_params_)

In [ ]:
x_model = xgb.XGBRegressor(
    n_estimators=513,
    max_depth=5,
    min_child_weight=3,
    gamma=0,
    colsample_bytree=0.9,nthread=5)

x_model.fit(x_train, y_train)
y_pred = x_model.predict(x_train)

rmse = np.sqrt(mean_squared_error(y_train, y_pred))
r2 = reg.score(x_train, y_train)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R^2 score is {}'.format(r2))
print("\n")